# Project3_1M_MovieLens Data Analysis

Author : Ashita Chandnani

Interesting trends -

[1. What are the k most popular movies of all times?](#section_1)

[2. What are the k most popular movies of a particular year?](#section_2)

[3. What are the k most popular movies by age group?](#section_3)

[4. What are the k movies with most ratings (presumably popular) with lowest ratings?](#section_4)

[5. What are the top k movies for the specified season?](#section_5)

[6. What is the average rating for each genre?](#section_6)

[7. What is the average rating given by each age group?](#section_7)

[8. What is the average age of users who rated movies in each genre?](#section_8)

[9. What are the top rated movies in each genre?](#section_9)

[10. Which genres are preferred by people with different occupational groups?](#section_10)

In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import *
import sys

In [2]:
# Creating a Spark session
spark = (SparkSession
        .builder
        .appName("Spark SQL Project3 Movie Data Mining Interesting Trends 1Million Dataset")
        .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/06 22:13:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Loading the DataFrames
movie_csv_file = "hdfs://cscluster00.boisestate.edu:9000/user/ashitachandnani/movies/ml-1m/movies.dat"
ratings_csv_file = "hdfs://cscluster00.boisestate.edu:9000/user/ashitachandnani/movies/ml-1m/ratings.dat"
users_csv_file = "hdfs://cscluster00.boisestate.edu:9000/user/ashitachandnani/movies/ml-1m/users.dat"

In [4]:
movies_df = (spark.read.format("csv")
      .option("sep", "::")
      .option("inferschema", "true")
      .option("samplingRatio", 0.1)  # Adjust the sampling ratio 
      .load(movie_csv_file)
      .toDF("MovieID","Titles","Genres"))

In [5]:
ratings_df = (spark.read.format("csv")
    .option("sep", "::")
    .option("inferschema", "true")
    .option("samplingRatio", 0.1)  # Adjust the sampling ratio
    .load(ratings_csv_file)
    .toDF("UserID", "MovieID", "Rating", "Timestamp"))

In [6]:
users_df = (spark.read.format("csv")
    .option("sep", "::")
    .option("inferschema", "true")
    .option("samplingRatio", 0.1)  # Adjust the sampling ratio
    .load(users_csv_file)
    .toDF("UserID", "Gender", "Age", "Occupation", "Zip-code"))

In [7]:
movies_df.show(5)

+-------+--------------------+--------------------+
|MovieID|              Titles|              Genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Animation|Childre...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|        Comedy|Drama|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [8]:
movies=movies_df.withColumn("Genre", explode(split(trim(col("Genres")), "\\|")))
movies.show(5)

+-------+----------------+--------------------+----------+
|MovieID|          Titles|              Genres|     Genre|
+-------+----------------+--------------------+----------+
|      1|Toy Story (1995)|Animation|Childre...| Animation|
|      1|Toy Story (1995)|Animation|Childre...|Children's|
|      1|Toy Story (1995)|Animation|Childre...|    Comedy|
|      2|  Jumanji (1995)|Adventure|Childre...| Adventure|
|      2|  Jumanji (1995)|Adventure|Childre...|Children's|
+-------+----------------+--------------------+----------+
only showing top 5 rows



In [9]:
movies=movies.drop('Genres')
movies.show(5)

+-------+----------------+----------+
|MovieID|          Titles|     Genre|
+-------+----------------+----------+
|      1|Toy Story (1995)| Animation|
|      1|Toy Story (1995)|Children's|
|      1|Toy Story (1995)|    Comedy|
|      2|  Jumanji (1995)| Adventure|
|      2|  Jumanji (1995)|Children's|
+-------+----------------+----------+
only showing top 5 rows



In [10]:
ratings_df.show(5)

+------+-------+------+---------+
|UserID|MovieID|Rating|Timestamp|
+------+-------+------+---------+
|     1|   1193|     5|978300760|
|     1|    661|     3|978302109|
|     1|    914|     3|978301968|
|     1|   3408|     4|978300275|
|     1|   2355|     5|978824291|
+------+-------+------+---------+
only showing top 5 rows



In [11]:
users_df.show(5)

+------+------+---+----------+--------+
|UserID|Gender|Age|Occupation|Zip-code|
+------+------+---+----------+--------+
|     1|     F|  1|        10|   48067|
|     2|     M| 56|        16|   70072|
|     3|     M| 25|        15|   55117|
|     4|     M| 45|         7|   02460|
|     5|     M| 25|        20|   55455|
+------+------+---+----------+--------+
only showing top 5 rows



In [12]:
# Joining ratings and users on UserID
movies_ratings_df = movies.join(ratings_df, on="MovieID")

In [13]:
# Joining with movies on MovieID
joined_df = movies_ratings_df.join(users_df, on="UserID")

In [14]:
# Convert the timestamp to a date and extract the month
joined_df = joined_df.withColumn("ReleaseMonth", month(from_unixtime("Timestamp")))

In [15]:
joined_df.createOrReplaceTempView("all_tbl")

In [16]:
spark.sql("Select * from all_tbl limit (5)").show()

+------+-------+--------------------+----------+------+---------+------+---+----------+--------+------------+
|UserID|MovieID|              Titles|     Genre|Rating|Timestamp|Gender|Age|Occupation|Zip-code|ReleaseMonth|
+------+-------+--------------------+----------+------+---------+------+---+----------+--------+------------+
|     1|   1193|One Flew Over the...|     Drama|     5|978300760|     F|  1|        10|   48067|          12|
|     1|    661|James and the Gia...|   Musical|     3|978302109|     F|  1|        10|   48067|          12|
|     1|    661|James and the Gia...|Children's|     3|978302109|     F|  1|        10|   48067|          12|
|     1|    661|James and the Gia...| Animation|     3|978302109|     F|  1|        10|   48067|          12|
|     1|    914| My Fair Lady (1964)|   Romance|     3|978301968|     F|  1|        10|   48067|          12|
+------+-------+--------------------+----------+------+---------+------+---+----------+--------+------------+



Now we can issue standard SQL queries. These would be no differenet than what we would
do in a standrad relational database.

The all_tbl has the data of all 3 tables joined on the common columns:
* MovieID
* UserID

Now we can proceed with answering all our questions using standard SQL statements

# Interesting Trends -

<a id="section_1"></a>
## 1. What are the k most popular movies of all times?

In [17]:
# SQL queries to find k most popular movies of all times

# Define the value of k (number of most popular movies we want to retrieve)
k = 10

result=spark.sql(f"""SELECT MovieID,Titles,Count(*) as RatingCount,
Round(AVG(Rating),2) AS AvgRating
FROM all_tbl
Group By MovieID, Titles
ORDER by RatingCount DESC,AvgRating DESC
LIMIT {k}""")
result.show()


+-------+--------------------+-----------+---------+
|MovieID|              Titles|RatingCount|AvgRating|
+-------+--------------------+-----------+---------+
|   1196|Star Wars: Episod...|      14950|     4.29|
|   1210|Star Wars: Episod...|      14415|     4.02|
|    260|Star Wars: Episod...|      11964|     4.45|
|   1580| Men in Black (1997)|      10152|     3.74|
|   1197|Princess Bride, T...|       9272|      4.3|
|   1617|L.A. Confidential...|       9152|     4.22|
|   1097|E.T. the Extra-Te...|       9076|     3.97|
|   2628|Star Wars: Episod...|       9000|     3.41|
|   1214|        Alien (1979)|       8096|     4.16|
|    480|Jurassic Park (1993)|       8016|     3.76|
+-------+--------------------+-----------+---------+



This global popularity analysis provides insights into the overall popularity of movies across all genres and years. It identifies movies that have received the highest number of ratings, indicating a broad appeal among users.
 
 Platforms can use this information to understand which movies have resonated the most with their user base. This information about the most popular movies of all time can be used for catalog promotion. 
 
Highlighting these movies can attract new users and retain existing ones by offering a selection of widely appreciated content.

**Star Wars** is most rated overall with more than 10k 4_and_up ratings.

<a id="section_2"></a>
## 2. What are the k most popular movies of a particular year?

In [18]:
# SQL query to find k most popular movies of a particular year

# Define the value of k (number of most popular movies we want to retrieve)
k = 10

# Define the desired year
target_year = 2002  

result=spark.sql(f"""SELECT MovieID,Titles,Count(*) as RatingCount,
Round(AVG(Rating),2) AS AvgRating
FROM all_tbl
WHERE YEAR(FROM_UNIXTIME(Timestamp))=={target_year}
Group By MovieID, Titles
ORDER by RatingCount DESC, AvgRating DESC
LIMIT {k}""")
result.show()


+-------+--------------------+-----------+---------+
|MovieID|              Titles|RatingCount|AvgRating|
+-------+--------------------+-----------+---------+
|   1196|Star Wars: Episod...|        195|     4.38|
|   2628|Star Wars: Episod...|        192|     3.02|
|   1210|Star Wars: Episod...|        170|     3.76|
|    260|Star Wars: Episod...|        164|     4.44|
|   1097|E.T. the Extra-Te...|        160|      4.0|
|   1200|       Aliens (1986)|        160|      3.8|
|   1214|        Alien (1979)|        148|     4.03|
|   1127|   Abyss, The (1989)|        144|     3.64|
|   3897|Almost Famous (2000)|        136|      3.9|
|   2692|Run Lola Run (Lol...|        135|     4.07|
+-------+--------------------+-----------+---------+



The query allows for an annual analysis of the most popular movies, helping content platforms understand the trends and preferences of users during a specific year (e.g., 2002 **Star wars** was most popular).
 
 Yearly user engagement metrics can help the platforms assess success of their content strategy and make informed decisions for the future.
 
 Users may find value in discovering or revisiting movies that were celebrated during a specific time frame.
 
 Highlighting the top movies of a given year can enhance user satisfaction and keep the catalog diverse and engaging.


<a id="section_3"></a>
## 3. What are the k most popular movies by age group?

In [19]:
# SQL Query to find k most popular movies by age group

# Define the value of k (number of most popular movies we want to retrieve)
k = 10

# Define the desired age group
target_age_group = 25   # it represents  "25-34" age bracket

result=spark.sql(f"""SELECT MovieID,Titles,Count(*) as RatingCount,
Round(AVG(Rating),2) AS AvgRating
FROM all_tbl
WHERE Age=={target_age_group}
Group By MovieID, Titles
ORDER by RatingCount DESC, AvgRating DESC
LIMIT {k}""")
result.show()


+-------+--------------------+-----------+---------+
|MovieID|              Titles|RatingCount|AvgRating|
+-------+--------------------+-----------+---------+
|   1196|Star Wars: Episod...|       5880|     4.39|
|   1210|Star Wars: Episod...|       5670|     4.06|
|    260|Star Wars: Episod...|       4512|     4.57|
|   1197|Princess Bride, T...|       3920|     4.37|
|   1580| Men in Black (1997)|       3884|     3.67|
|   2628|Star Wars: Episod...|       3612|     3.37|
|   1097|E.T. the Extra-Te...|       3544|     3.96|
|   1617|L.A. Confidential...|       3520|     4.24|
|   2916| Total Recall (1990)|       3416|     3.65|
|    589|Terminator 2: Jud...|       3261|     4.02|
+-------+--------------------+-----------+---------+



By focusing on a specific age group (e.g., in 25-34 ages,  **Star Wars** is most popular), the query aims to provide personalized recommendations tailored to the preferences of users within that age range. 
 
This is particularly relevant for content platforms seeking to enhance user experience and engagement.
 
 Users are more likely to return to a platform that consistently offers content that aligns with their preferences.

<a id="section_4"></a>
## 4. What are the k movies with most ratings (presumably popular) with lowest ratings?

In [20]:
# Define the value of k (number of most popular movies we want to retrieve)
k = 10

# SQL queries to find k movies with most ratings (presumably popular) with lowest Ratings
result=spark.sql(f"""SELECT MovieID,Titles,Round(AVG(Rating),2) as AvgRating,Count(*) as RatingCount
FROM all_tbl
Group By MovieID, Titles
HAVING RatingCount > 1  -- To exclude movies with only one rating
ORDER by RatingCount DESC, AvgRating ASC
LIMIT {k}""")
result.show()

+-------+--------------------+---------+-----------+
|MovieID|              Titles|AvgRating|RatingCount|
+-------+--------------------+---------+-----------+
|   1196|Star Wars: Episod...|     4.29|      14950|
|   1210|Star Wars: Episod...|     4.02|      14415|
|    260|Star Wars: Episod...|     4.45|      11964|
|   1580| Men in Black (1997)|     3.74|      10152|
|   1197|Princess Bride, T...|      4.3|       9272|
|   1617|L.A. Confidential...|     4.22|       9152|
|   1097|E.T. the Extra-Te...|     3.97|       9076|
|   2628|Star Wars: Episod...|     3.41|       9000|
|   1214|        Alien (1979)|     4.16|       8096|
|    480|Jurassic Park (1993)|     3.76|       8016|
+-------+--------------------+---------+-----------+



The combination of high rating counts and lower average ratings may indicate that certain movies are widely watched but not universally well-received. 
 
 Understanding user engagement with such movies can be crucial for content platforms to refine their recommendation algorithms and content curation strategies.
 
 It present an opportunity for content platforms to analyze user feedback and identify areas for improvement. 
 
 It may also be useful in balancing content recommrndations. While some users may enjoy popular blockbusters, others may prefer movies with niche appeal or unconventional storytelling.


<a id="section_5"></a>
## 5. What are the top k movies for the specified season?

In [21]:
# SQL query to find the top k movies for the specified season

# Define the value of k (number of most popular movies we want to retrieve)
k = 10

# Define the season (1: Winter, 2: Spring, 3: Summer, 4: Fall)
target_season = 3  # Example: 3 for Summer

result=spark.sql(f"""SELECT MovieID,Titles,Round(AVG(Rating),2) as AvgRating,Count(*) as RatingCount
FROM all_tbl
WHERE ReleaseMonth >= {target_season * 3 - 2} AND ReleaseMonth <= {target_season * 3}
Group By MovieID, Titles
HAVING RatingCount > 1  -- To exclude movies with only one rating
ORDER by RatingCount DESC, AvgRating ASC
LIMIT {k}""")
result.show()


+-------+--------------------+---------+-----------+
|MovieID|              Titles|AvgRating|RatingCount|
+-------+--------------------+---------+-----------+
|   1196|Star Wars: Episod...|     4.32|       4935|
|   1210|Star Wars: Episod...|     4.05|       4875|
|    260|Star Wars: Episod...|     4.51|       4008|
|   1197|Princess Bride, T...|     4.34|       3284|
|   1580| Men in Black (1997)|     3.81|       3200|
|   1097|E.T. the Extra-Te...|     3.92|       3056|
|   1617|L.A. Confidential...|      4.2|       3056|
|   2628|Star Wars: Episod...|     3.46|       2964|
|   1214|        Alien (1979)|     4.16|       2864|
|   2916| Total Recall (1990)|     3.64|       2824|
+-------+--------------------+---------+-----------+



This SQL query finds the top k movies for a specified season.
 
Understanding seasonal viewing patterns can be used for seasonal content recommendation. 

<a id="section_6"></a>
## 6. What is the average rating for each genre?

In [22]:

# SQL query to calculate the average rating for each genre
result=spark.sql("""SELECT Genre,Round(AVG(Rating),2) as AvgRating
FROM 
    (SELECT MovieID,Rating,Genre
    FROM all_tbl)
Group By Genre
ORDER by AvgRating DESC
""")
result.show()


+-----------+---------+
|      Genre|AvgRating|
+-----------+---------+
|  Film-Noir|     4.08|
|Documentary|     3.93|
|        War|     3.89|
|      Drama|     3.77|
|      Crime|     3.71|
|  Animation|     3.68|
|    Mystery|     3.67|
|    Musical|     3.67|
|    Western|     3.64|
|    Romance|     3.61|
|   Thriller|     3.57|
|     Comedy|     3.52|
|     Action|     3.49|
|  Adventure|     3.48|
|     Sci-Fi|     3.47|
|    Fantasy|     3.45|
| Children's|     3.42|
|     Horror|     3.22|
+-----------+---------+



This query calculates the average rating for each genre by aggregating the ratings of movies within each genre.
 
This information facilitates data-driven decision-making in the entertainment industry by providing insights into the popularity and user perceptions of different genres. 
 
Platforms can leverage this information to enhance content curation, user engagement, and overall strategic planning.

**Film-Noir** stands out with highest average rating and **Horror** with the lowest average rating.

<a id="section_7"></a>
## 7. What is the average rating given by each age group?

In [23]:
# SQL query to calculate the average rating by age group
result=spark.sql("""SELECT Age,Round(AVG(Rating),2) as AvgRating
FROM all_tbl
Group By Age
ORDER by Age ASC
""")
result.show()


+---+---------+
|Age|AvgRating|
+---+---------+
|  1|     3.54|
| 18|     3.51|
| 25|     3.55|
| 35|     3.62|
| 45|     3.64|
| 50|     3.72|
| 56|     3.76|
+---+---------+



This query is useful for understanding the average ratings given by users in different age groups. 
 
Analyzing the average ratings based on age can provide valuable insights into the preferences and satisfaction levels of users across different age demographics.

**Older age groups** tend to give more average ratings.

<a id="section_8"></a>
## 8. What is the average age of users who rated movies in each genre?

In [24]:
# SQL query to calculate the average age of users who rated movies in each genre
result=spark.sql("""SELECT Genre,Round(AVG(Age),2) as AvgAge
FROM all_tbl
Group By Genre
ORDER by AvgAge DESC
""")
result.show()


+-----------+------+
|      Genre|AvgAge|
+-----------+------+
|    Western| 32.83|
|  Film-Noir| 32.62|
|        War|  31.4|
|    Mystery| 31.13|
|      Drama| 30.75|
|Documentary| 30.32|
|    Romance| 30.23|
|    Musical| 30.03|
|     Sci-Fi| 29.44|
|      Crime| 29.42|
|   Thriller|  29.4|
|  Adventure| 29.24|
|     Action| 29.12|
|     Comedy| 29.04|
|     Horror| 28.76|
|    Fantasy| 28.27|
| Children's| 27.51|
|  Animation| 27.14|
+-----------+------+




 This query is useful to the platform for understanding the average age of users who rate movies in different genres. 
 
 It provides insights into the age demographics of the audience for each genre, which can be valuable for optimizing various aspects of content strategy and user engagements.
 
 Average age of people rating **Western** movies is about 32 years.

<a id="section_9"></a>
## 9. What are the top rated movies in each genre?

In [25]:
# SQL query to find top rated movies in each genre
result=spark.sql("""SELECT
    Genre,
    MovieID,
    Titles,
    Round(AVG(Rating),2) AS AvgRating,
    COUNT(*) AS RatingCount,
    RANK() OVER (PARTITION BY Genre ORDER BY AVG(Rating) DESC,COUNT(*) DESC) AS Ranking
FROM
    all_tbl
GROUP BY
    Genre, MovieID,Titles
ORDER BY
    Genre, Ranking;
""")
result.show(20)

+------+-------+--------------------+---------+-----------+-------+
| Genre|MovieID|              Titles|AvgRating|RatingCount|Ranking|
+------+-------+--------------------+---------+-----------+-------+
|Action|   2905|      Sanjuro (1962)|     4.61|         69|      1|
|Action|   2019|Seven Samurai (Th...|     4.56|        628|      2|
|Action|    858|Godfather, The (1...|     4.52|       2223|      3|
|Action|   1198|Raiders of the Lo...|     4.48|       2514|      4|
|Action|    260|Star Wars: Episod...|     4.45|       2991|      5|
|Action|   1221|Godfather: Part I...|     4.36|       1692|      6|
|Action|   2028|Saving Private Ry...|     4.34|       2653|      7|
|Action|   2571|  Matrix, The (1999)|     4.32|       2590|      8|
|Action|   1197|Princess Bride, T...|      4.3|       2318|      9|
|Action|   1233|Boat, The (Das Bo...|      4.3|       1001|     10|
|Action|   1196|Star Wars: Episod...|     4.29|       2990|     11|
|Action|    969|African Queen, Th...|     4.25| 

The above query calculates the average rating, rating count, and ranking for each movie by genre. It uses the RANK() window function to rank movies within each genre based on the average rating and count of ratings.
 
 This information can be useful top recommend top rated movies in each genre to users who show an interest in a specific genre.


**Sanjuro** and **Seven Samurai** are have highest average ratings in Action Genre while **Star Wars** still remains to have highest number of ratings.

<a id="section_10"></a>
## 10. Which genres are preferred by people with different occupational groups?

In [26]:
# SQL query to find which genres are preferred by people with different occupational groups?
result=spark.sql("""
WITH OccupationGenres AS (
    SELECT
        UserID,
        MovieID,
        Titles,
        Genre,
        Rating,
        Occupation,
        CASE
            WHEN Occupation = 0 THEN 'other'
            WHEN Occupation = 1 THEN 'academic/educator'
            WHEN Occupation = 2 THEN 'artist'
            WHEN Occupation = 3 THEN 'clerical/admin'
            WHEN Occupation = 4 THEN 'college/grad student'
            WHEN Occupation = 5 THEN 'customer service'
            WHEN Occupation = 6 THEN 'doctor/health care'
            WHEN Occupation = 7 THEN 'executive/managerial'
            WHEN Occupation = 8 THEN 'farmer'
            WHEN Occupation = 9 THEN 'homemaker'
            WHEN Occupation = 10 THEN 'K-12 student'
            WHEN Occupation = 11 THEN 'lawyer'
            WHEN Occupation = 12 THEN 'programmer'
            WHEN Occupation = 13 THEN 'retired'
            WHEN Occupation = 14 THEN 'sales/marketing'
            WHEN Occupation = 15 THEN 'scientist'
            WHEN Occupation = 16 THEN 'self-employed'
            WHEN Occupation = 17 THEN 'technician/engineer'
            WHEN Occupation = 18 THEN 'tradesman/craftsman'
            WHEN Occupation = 19 THEN 'unemployed'
            WHEN Occupation = 20 THEN 'writer'
            ELSE 'unknown'
        END AS OccupationGroup
    FROM
        all_tbl)
SELECT
    OccupationGroup,
    Genre,
    ROUND(AVG(Rating),2) AS AvgRating
FROM
    OccupationGenres
GROUP BY
    OccupationGroup, Genre
ORDER BY
    OccupationGroup, AvgRating DESC

""")
result.show(40)


+-----------------+-----------+---------+
|  OccupationGroup|      Genre|AvgRating|
+-----------------+-----------+---------+
|     K-12 student|  Film-Noir|     4.21|
|     K-12 student|        War|     3.88|
|     K-12 student|      Drama|     3.78|
|     K-12 student|      Crime|     3.69|
|     K-12 student|    Mystery|     3.64|
|     K-12 student|    Romance|     3.62|
|     K-12 student|Documentary|     3.58|
|     K-12 student|    Musical|     3.56|
|     K-12 student|   Thriller|     3.55|
|     K-12 student|    Western|     3.51|
|     K-12 student|     Action|      3.5|
|     K-12 student|     Comedy|      3.5|
|     K-12 student|  Animation|     3.46|
|     K-12 student|     Sci-Fi|     3.44|
|     K-12 student|  Adventure|     3.43|
|     K-12 student|    Fantasy|      3.3|
|     K-12 student|     Horror|     3.24|
|     K-12 student| Children's|     3.22|
|academic/educator|  Film-Noir|     4.08|
|academic/educator|Documentary|     3.98|
|academic/educator|        War|   

The above query provides information on the preferred genres for different occupation groups based on the average ratings given by users in those groups.
 
 This information can be used to provide personalized content suggestions, user engagement and marketing strategies for users in specific professions and improve overall user experience in diverse occuptional groups.
 
**Film-Noir** and **Documentary** have highest average ratings in the academic/educator group.

In [27]:
spark.stop()